## Sentiment Analysis Tool

In [ ]:
# Using Textblob to analyze sentiment example
from textblob import TextBlob

text = "I THINK dodgecoin is great! I'm so excited to buy more!"
blob = TextBlob(text)
sentiment = blob.sentiment

print(f"Polarity: {sentiment.polarity}, Subjectivity: {sentiment.subjectivity}")


# Polarity is a float within the range [-1.0, 1.0], where -1 means negative sentiment and 1 means positive sentiment.
# Subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.


## Twitter API

In [ ]:
import requests
import os
from dotenv import load_dotenv


load_dotenv()
# Now you can access your environment variable securely
bearer_token = os.getenv('TWITTER_BEARER_TOKEN')



def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url():
    hashtag = "%23crypto"
    exclude_phrases = "-\"signing up\" -\"receive tokens\" -giveaway"
    additional_keywords = "\"crypto\" OR \"cryptocurrency\""
    language = "lang:en"
    # Ensure proper spacing and logical grouping
    query = f"{hashtag} {exclude_phrases} {additional_keywords} {language}"
    # Forming the URL
    url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&tweet.fields=author_id,created_at"
    return url


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()



In [ ]:
# Set up the URL and headers
url = create_url()
headers = create_headers(bearer_token)

# Make the request
json_response = connect_to_endpoint(url, headers)

# Print the response
print(json_response)


In [ ]:
# storing tweet
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('crypto_tweets.db')
c = conn.cursor()

# Create a table to store tweets
c.execute('''CREATE TABLE IF NOT EXISTS tweets
             (id TEXT PRIMARY KEY, author_id TEXT, created_at TEXT, text TEXT)''')

conn.commit()



In [ ]:
def insert_tweet_data(json_response):
    tweets = json_response.get('data', [])
    for tweet in tweets:
        c.execute('INSERT OR IGNORE INTO tweets (id, author_id, created_at, text) VALUES (?, ?, ?, ?)',
                  (tweet['id'], tweet['author_id'], tweet['created_at'], tweet['text']))

    conn.commit()

insert_tweet_data(json_response)


In [ ]:
c.execute('SELECT * FROM tweets')
for row in c.fetchall():
    print(row)


In [ ]:
c.execute('SELECT COUNT (*) FROM tweets')
print(c.fetchone()[0])

In [ ]:
import pandas as pd
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('crypto_tweets.db')

# SQL query to select all tweets
query = "SELECT * FROM tweets"

# Load the results of the query into a DataFrame
tweets_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Now you can work with the tweets_df DataFrame
print(tweets_df.head())


In [ ]:
# Example: Count tweets by author_id
tweet_counts = tweets_df['author_id'].value_counts()
print(tweet_counts.head())

# Example: Simple sentiment analysis (assuming you have a 'sentiment' column)
# You might need to calculate this column beforehand
if 'sentiment' in tweets_df.columns:
    average_sentiment = tweets_df['sentiment'].mean()
    print(f"Average Sentiment: {average_sentiment}")


In [ ]:
tweets_df['text'] = tweets_df['text'].astype(str)
print(tweets_df['text'][50])

In [ ]:
from textblob import TextBlob


tweet = tweets_df['text'].iloc[-1]
print(tweet)

tweet_sentiment = TextBlob(tweet).sentiment
print(tweet_sentiment)

In [ ]:
import praw
from dotenv import load_dotenv
import os

## Reddit API

In [ ]:
import praw
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access environment variables
client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')
username = os.getenv('REDDIT_USERNAME')
password = os.getenv('REDDIT_PASSWORD')



# Initialize PRAW with your credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=	client_secret,
                     user_agent=user_agent,
                     username= username,
                     password= password)
print('Done')


In [ ]:
subreddit = reddit.subreddit("crypto")

for post in subreddit.top(limit=10):
    print(post.title, post.url)
